In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
sys.path.insert(0, '../')
from humannotator.humannotator import Annotator
from humannotator.data import Data_DataFrame, Data
from humannotator.annotations import Task_MultipleChoice, Annotations
import pandas as pd

In [ ]:
df = pd.read_csv('news.csv', index_col=0)
data = Data_DataFrame(df, items_col='title', id_col='news_id')

In [ ]:
instruction = "Choose one of the following options:"
choices={
    '0': 'not adverse media',
    '1': 'adverse media',
    '3': 'exclude from dataset'
}

task = Task_MultipleChoice(instruction=instruction, choices=choices)
annotations = Annotations(task)
annotator = Annotator(data, annotations)

In [ ]:
annotator(data.ids)

In [ ]:
annotator.annotations()

In [ ]:
annotator.annotations.to_dataframe()

In [ ]:
df_annotations = annotator.annotations.to_dataframe()
annotations = Annotations.from_dataframe(df_annotations)

In [ ]:
annotations()

In [ ]:
annotations.task